Node creation for ROBDD.Low means the final logic 0 and high means logic 1 


In [63]:
class Node:
    def __init__(self, name = None):
        self.name = name #name of the variable (a,b,c etc..)
        # high and low pointers
        self.high = None 
        self.low = None
        return
# low means logic 0 and high means logic 1
low=Node("0")
high=Node("1")

Unique Table for storing the variable pointers. 

In [64]:
def get_pointer_from_UT(key,node_ptr):
  if key in unique_table.keys(): #key found for pointers for variables
    return unique_table[key]
  else: #not found
    unique_table[key]=node_ptr
    return unique_table[key]

COFACTOR Function which computes the cofactor of a given function for a set of variables

In [67]:
def cofactor_function(function,variable):
  # the input is taken as a string (ab+c') and is splitted to form a list
    cubelist=function.split("+")
    for i in range (0,len(cubelist)):
        cubelist[i]=validate_cube(cubelist[i])
    var_list=get_variable_list(validate_cube(variable))
    cubelist=validate_cubelist(cubelist)
    cofactor=[]
    for var in var_list:
        cofactor=[]
        for cube in cubelist:
            if "'" in var:
                cofactor_var=cofactor_cube(cube,var[0],0)
                cofactor.append(cofactor_var)
            else:
                cofactor_var=cofactor_cube(cube,var[0],1)
                cofactor.append(cofactor_var)
        cubelist=cofactor
    # print(cofactor)
    for i in cofactor:
        if i=="1":
            return "1"
    for i in cofactor:
        if i=="0" and len(cofactor)>1:
            cofactor.remove(i)
    cofactor=validate_cubelist(cofactor)
    # final=""
    final="+".join(cofactor)
      # ""
    # print(final)
    return final # string format



def cofactor_cube(cube,literal,value):
  # """this function computes the cofactor for a cube """
    if cube=="1" or cube=="0":
        return cube
    else:
        var_list=get_variable_list(cube)
        for i in var_list:
            if literal==i[0]:
                if ("'" not in i and value==0) or ("'" in i and value==1):
                    return "0"
                elif ("'" not in i and value==1) and (len(cube)==1):
                    return "1"
                elif ("'" not in i and value==1) and len(cube)!=1:
                    var_list.remove(i)
                    break
                elif ("'" in i and value==0) and len(cube)==2:
                    return "1"
                elif ("'" in i and value==0) and len(cube)!=2:
                    var_list.remove(i)
                    break
        cofactor=""
        for i in var_list:
            cofactor="".join([cofactor,i])
        return cofactor



def validate_cubelist(cubelist):
  # """This function validates the cube list i.e removes redundancy etc..""" 
    cube_dict={}
    for cube in cubelist:
        if cube not in cube_dict:
            cube_dict[cube]=1
        else:
            cubelist.remove(cube)
    return cubelist



def validate_cube(cube):
  # Function validates the cube 
    literal_list_1=get_variable_list(cube)
    literal_dict_1={}
    for i in literal_list_1:
        if i not in literal_dict_1:
            literal_dict_1[i]=1
    cube_1=""
    for key in literal_dict_1:
        cube_1="".join([cube_1,key])
    literal_list_2=get_variable_list(cube_1)
    literal_dict_2={}
    for i in literal_list_2:
        if i[0] in literal_dict_2:
            return "0"
        else:
            literal_dict_2[i[0]]=1
        
    return cube_1
            
    
def get_variable_list(variable):
  # given a string of variables returns the list of variables
    length=len(variable)
    i=0
    final=[]
    while (i<length):
        if i<length-1:
            if variable[i]!="'" and variable[i+1]=="'":
                final.append(variable[i]+"'")
                i=i+2
            else:
                final.append(variable[i])
                i=i+1
        elif i==length-1:
            final.append(variable[i])
            i=i+1
    return final
      

ITE operator function which uses recursive calls and also Computed Table to avoid extra computation

In [68]:
def ITE_operator(function,variable_order,index):
  # computes the ITE which is the main part.
  variable=variable_order[index] # top variable
  variable_compliment=variable+"'" # compliment
  # Both cofactors are calculated
  cofactor_high=cofactor_function(function,variable)
  cofactor_low=cofactor_function(function,variable_compliment)
  #creating a key for variable in UT 
  key=variable+cofactor_high+cofactor_low
  #create a node for the variable
  node=Node(variable)
  #Edge cases
  if cofactor_high=="1":
    node.high=high
  if cofactor_high=="0":
    node.high=low
  if cofactor_low=="1":
    node.low=high
  if cofactor_low=="0":
    node.low=low
  #creating the keys for Computed table
  CT_key_high=cofactor_high+str(index+1)
  CT_key_low=cofactor_low+str(index+1)
  if node.high==None:
    if CT_key_high not in computed_table: #check
      node.high=ITE_operator(cofactor_high,variable_order,index+1)
      computed_table[CT_key_high]=node.high
    else:
      node.high=computed_table[CT_key_high]
  if node.low==None:
    if CT_key_low not in computed_table: #check
      node.low=ITE_operator(cofactor_low,variable_order,index+1)
      computed_table[CT_key_low]=node.low
    else:
      node.low=computed_table[CT_key_low]
  node=get_pointer_from_UT(key,node) #check and update in UT 
  if node.low==node.high:
    return node.low
  return node



INORDER TRAVERSAL FOR ROBDD 

In [69]:
def inorder_robdd(node):
  if node:
    inorder_robdd(node.low)
    print(node.name)
    inorder_robdd(node.high)
    

Main function calls.


In [71]:
unique_table={}
computed_table={}
function=input("enter the string of sops  ")
variable_order_str=input("enter the variable order  ")
variable_order=variable_order_str.split(" ")
print(variable_order)
robdd=ITE_operator(function,variable_order,0)
s=input("enter 1 for inorder traversal and 0 to display the tree ")
COUNT=[10]
if s=="0":
  print2D(robdd)
else:
  inorder_robdd(robdd)



enter the string of sops  a+bc
enter the variable order  b c a
['b', 'c', 'a']
enter 1 for inorder traversal and 0 to display the tree 1
0
a
1
b
0
a
1
c
1


PRINTING TREE FUNCTION . TAKEN FROM INTERNET

In [57]:
def print2DUtil(root, space) :
 
    # Base case
    if (root == None) :
        return
 
    # Increase distance between levels
    space += COUNT[0]
 
    # Process right child first
    print2DUtil(root.high, space)
 
    # Print current node after space
    # count
    print()
    for i in range(COUNT[0], space):
        print(end = " ")
    print(root.name)
 
    # Process left child
    print2DUtil(root.low, space)
 
# Wrapper over print2DUtil()
def print2D(root) :
     
    # space=[0]
    # Pass initial space count as 0
    print2DUtil(root, 0)